In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import seaborn as sns
from scipy import integrate
import numpy as np
import jax
import jax.numpy as jnp

from dosipy.constants import eps_0
from dosipy.field import poynting
from dosipy.utils.dataloader import (load_tissue_diel_properties,
                                     load_antenna_el_properties)
from dosipy.utils.integrate import elementwise_quad, elementwise_rectquad
from dosipy.utils.derive import holoborodko

from utils import reflection_coefficient

# Input data

In [ ]:
# working frequency, Hz
f = 60e9

# dry skin density, kg/m3
rho = 1109

# conductivity, relative permitivitya and penetration depth
sigma, eps_r, _, pen_depth = load_tissue_diel_properties('skin_dry', f)
L = pen_depth / 2  # energy penetration depth in m

# reflection coefficient
eps_i = sigma / (2 * np.pi * f * eps_0)
eps = eps_r - 1j * eps_i
gamma = reflection_coefficient(eps)

# power transmission coefficient
T_tr = 1 - gamma ** 2

# Solution domain setup

In [ ]:
# exposed surface extent
if f < 30e9:
    exposure_extent = (0.02, 0.02)  # meters x meters
else:
    exposure_extent = (0.01, 0.01)  # meters x meters

# exposed surface area
A = exposure_extent[0] * exposure_extent[1]  # meters^2

# exposed volume coordinates
xt = jnp.linspace(-exposure_extent[0]/2, exposure_extent[0]/2)
yt = jnp.linspace(-exposure_extent[1]/2, exposure_extent[1]/2)
Xt, Yt = np.meshgrid(xt, yt)
z_max = 0.02  # in meters
zt = jnp.linspace(0, z_max)

# distance from the antenna
h = 5 / 1000  # meters

# Current across dipole in free space

In [ ]:
# antenna electric properties, free space (Poljak 2005)
antenna_data = load_antenna_el_properties(f)

# antenna position - coordinates
xs = antenna_data.x.to_numpy()
xs = xs - xs.max() / 2
xs = jnp.asarray(xs)
ys = jnp.zeros_like(xs)
zs = jnp.full_like(xs, h)

# current through the antenna
Is = antenna_data.ireal.to_numpy() + antenna_data.iimag.to_numpy() * 1j

# current gradients
Is_x = holoborodko(Is, xs[1]-xs[0])

In [ ]:
sns.set_theme(style='ticks', font_scale=1.25, palette='colorblind')
fig, ax = plt.subplots()
width = (np.ptp(xs) / np.ptp(xt) * 100).item()
axin = inset_axes(ax, width=f'{width}%', height='20%', loc='center',
                  bbox_to_anchor=(0, 0.1, 1, 1), bbox_transform=ax.transAxes)

# main axes, exposed surface
bbox = [xt.min(), xt.max(), yt.min(), yt.max()]
xmin, xmax = bbox[:2]
ymin, ymax = bbox[2:]
ax.vlines(x=bbox[:2], ymin=ymin, ymax=ymax,
          color='k', lw=2)
ax.hlines(y=bbox[2:], xmin=xmin, xmax=xmax,
          color='k', lw=2)
ax.hlines(y=ys[0], xmin=xs.min(), xmax=xs.max(),
          color='r', lw=4)
ax.set_aspect('equal', 'box')
ax.set(xlabel='$x$ (cm)',
       ylabel='$y$ (cm)',
       xticks=[xmin, xs.min(), xs.max(), xmax],
       yticks=[ymin, 0, ymax],
       xticklabels=[xmin * 100,
                    np.round(xs.min() * 100, 2),
                    np.round(xs.max() * 100, 2),
                    xmax * 100],
       yticklabels=[ymin * 100, 0, ymax * 100])
sns.despine(fig=fig, ax=ax)

# inserted axes, antenna position relative to surface
axin.plot(xs, np.abs(Is))
axin.set(xlabel='dipole',
         ylabel='|$I$| (mA)',
         xticks=[xs.min(), xs.max()],
         yticks=[0, np.abs(Is).max()],
         xticklabels=[],
         yticklabels=[0, np.round(np.abs(Is).max() * 1000, 1)])
axin.patch.set_alpha(0)

# EM field distribution in free space

In [ ]:
# incident PD components on the exposed surface
PDinc = np.empty((xt.size, yt.size, 3), dtype=np.complex128)
for xi, _xt in enumerate(xt):
    for yi, _yt in enumerate(yt):
        PDinc[xi, yi, :] = poynting(_xt, _yt, zt[0], xs, ys, zs, f, Is, Is_x)
        
# averaged total and normal incident power density on the surface
PDinc_tot = np.abs(np.sqrt(PDinc[:, :, 0] ** 2
                           + PDinc[:, :, 1] ** 2
                           + PDinc[:, :, 2] **2))
PDinc_n = np.abs(PDinc[:, :, 1].real)

# Specific absorption rate within the exposed tissue

In [ ]:
# specific absorbtion rate in the exposed volume
sar_surf = PDinc_tot * T_tr / (rho * L)
sar = np.outer(
    sar_surf, np.exp(-zt / L)
).reshape(*sar_surf.shape, *zt.shape)

In [ ]:
sns.set_theme(style='ticks', font_scale=1.25, palette='colorblind')
fig, ax = plt.subplots()

# main axes, exposed surface
cm = ax.imshow(sar_surf,
               extent=bbox,
               origin='lower',
               interpolation='bicubic',
               cmap='viridis')
cb = fig.colorbar(cm, ax=ax,
                  label='SAR (W$/$m$^3$)')
ax.hlines(y=ys[0], xmin=xs.min(), xmax=xs.max(),
          color='r', lw=4)
ax.set(xlabel='$x$ (cm)',
       ylabel='$y$ (cm)',
       xticks=[xmin, xs.min(), xs.max(), xmax],
       yticks=[ymin, 0, ymax],
       xticklabels=[xmin * 100,
                    np.round(xs.min() * 100, 2),
                    np.round(xs.max() * 100, 2),
                    xmax * 100],
       yticklabels=[ymin * 100, 0, ymax * 100])
sns.despine(fig=fig, ax=ax)

# Spatially averaged abosorbed power density

## 1) via tranmistted power

In [ ]:
# compute transmitted power density and its coresponding surface average
PD_ab_tot = np.zeros_like(sar_surf)
for i in range(sar_surf.shape[0]):
    for j in range(sar_surf.shape[1]):
        PD_ab_tot[i, j] = integrate.trapezoid(sigma * sar[i, j, :], zt, dx=np.diff(zt))
sPDab_tot = 1 / (2 * A) * elementwise_rectquad(xt, yt, PD_ab_tot)

In [ ]:
sns.set_theme(style='ticks', font_scale=1.25, palette='colorblind')
fig, ax = plt.subplots()

# main axes, exposed surface
cm = ax.imshow(PD_ab_tot,
               extent=bbox,
               origin='lower',
               interpolation='bicubic',
               cmap='viridis')
cb = fig.colorbar(cm, ax=ax,
                  label='TPD (W$/$m$^2$)')
ax.hlines(y=ys[0], xmin=xs.min(), xmax=xs.max(),
          color='r', lw=4)
ax.set(xlabel='$x$ (cm)',
       ylabel='$y$ (cm)',
       xticks=[xmin, xs.min(), xs.max(), xmax],
       yticks=[ymin, 0, ymax],
       xticklabels=[xmin * 100,
                    np.round(xs.min() * 100, 2),
                    np.round(xs.max() * 100, 2),
                    xmax * 100],
       yticklabels=[ymin * 100, 0, ymax * 100],
       title=f'APD = {sPDab_tot:.2f} W$/$m$^2$')
sns.despine(fig=fig, ax=ax)

## 2) flux defintion

In [ ]:
sPDinc_n = 1 / (2 * A) * elementwise_rectquad(xt, yt, PDinc_n)
sPDapd_n = T_tr * sPDinc_n
sPDapd_n